# Building the Core Dataset for Plant Recommendations (Table13)

Name: Zihan

## Step 1: 导入所需库并设置文件路径
首先，我们导入所有需要的 Python 库，并定义好存放原始JSON文件的目录和最终输出CSV文件的路径。

In [6]:
# 导入所需库
import os
import json
import pandas as pd
from glob import glob

# 定义输入（原始数据）和输出（处理后数据）的路径
DETAILS_DIR = "01_raw_data/01_species_details"
OUTPUT_PATH = "02_wrangled_data/Table13_GeneralPlantListforRecommendation.csv"

## Step 2: 定义核心数据 - 耐寒区(Hardiness Zone)到温度(°C)的转换表
这是我们实现温度转换的关键。我们创建一个字典（lookup table），用于将植物的最低耐寒区直接转换为它能生存的绝对最低温度（摄氏度）。

In [7]:
# 该转换表基于美国农业部(USDA)的植物耐寒区标准
HARDINESS_ZONE_TO_CELSIUS = {
    "1": -51.1, "2": -45.6, "3": -40.0, "4": -34.4, "5": -28.9,
    "6": -23.3, "7": -17.8, "8": -12.2, "9": -6.7, "10": -1.1,
    "11": 4.4,  "12": 10.0, "13": 15.6
}

## Step 3: 定义处理单个JSON文件的函数
为了让代码更整洁，我们创建一个函数，专门负责处理单个植物的JSON文件。它的任务是读取JSON内容，提取我们需要的字段，并完成从耐寒区到具体温度的转换。

In [8]:
def process_plant_details(details_json):
    """
    从单个植物的JSON数据中提取并转换所需信息。
    (已更新版本，增加了list到JSON字符串的转换)
    """
    # 根据原始逻辑，跳过ID大于3000的植物
    plant_id = details_json.get("id")
    if not plant_id or plant_id > 3000:
        return None

    # --- 将耐寒区(Hardiness Zone)转换为绝对最低温度 ---
    hardiness_data = details_json.get("hardiness", {})
    min_zone = hardiness_data.get("min")
    # 从转换表中查找温度，如果找不到则默认为None
    absolute_min_temp = HARDINESS_ZONE_TO_CELSIUS.get(min_zone)

    # --- 健壮地处理plant_type字段 ---
    plant_type_raw = details_json.get("type")
    plant_type_processed = plant_type_raw.lower() if plant_type_raw else ""

    # --- 构建我们需要的记录 ---
    record = {
        "general_plant_id": plant_id,
        "plant_type": plant_type_processed,
        # 【修改之处】直接将sunlight列表转换为JSON字符串，以便后续存入MySQL
        "sunlight": json.dumps(details_json.get("sunlight", []), ensure_ascii=False),
        "watering": details_json.get("watering"),
        "drought_tolerant": details_json.get("drought_tolerant", False),
        "absolute_min_temp_c": absolute_min_temp
    }
    return record

## Step 4: 主流程 - 遍历文件、处理数据并创建DataFrame
现在，我们将执行主要的数据处理流程。代码会遍历所有JSON文件，使用上面定义的函数来处理它们，然后将所有结果汇总到一个 pandas DataFrame 中。

In [17]:
print("开始处理所有植物的JSON文件...")

# 查找所有植物详情的JSON文件
detail_files = glob(os.path.join(DETAILS_DIR, "plant_species_details_*.json"))
print(f"找到了 {len(detail_files)} 个JSON文件准备处理。")

# 循环遍历所有文件并处理
all_plant_data = []
for file_path in detail_files:
    try:
        with open(file_path, "r", encoding="utf-8") as f:
            data = json.load(f)
            processed_record = process_plant_details(data)
            # 只有成功处理的记录才会被添加
            if processed_record:
                all_plant_data.append(processed_record)
    except Exception as e:
        print(f"处理文件 {file_path} 时发生错误: {e}")

print(f"成功处理了 {len(all_plant_data)} 条植物记录。")

# 将记录列表转换为pandas DataFrame，方便后续操作
df = pd.DataFrame(all_plant_data)
df = df.sort_values('general_plant_id')

print("DataFrame创建成功，准备进行筛选。")
df.head() # 显示前几行数据检查一下

开始处理所有植物的JSON文件...
找到了 1484 个JSON文件准备处理。
成功处理了 1484 条植物记录。
DataFrame创建成功，准备进行筛选。


,general_plant_id,plant_type,sunlight,watering,drought_tolerant,absolute_min_temp_c
0,1,tree,"[""full sun""]",Frequent,False,-17.8
596,2,tree,"[""full sun""]",Average,False,-34.4
707,3,tree,"[""Full sun"", ""part shade""]",Average,True,-40.0
818,4,tree,"[""full sun""]",Average,True,-34.4
929,5,tree,"[""full sun"", ""part shade"", ""filtered shade""]",Frequent,False,-23.3


## Step 5: 筛选数据、整理格式并保存为CSV文件
最后一步，我们对 DataFrame 进行筛选（剔除树木），整理列的顺序和格式，然后将最终完美的结果保存为CSV文件。

In [18]:
# --- 筛选步骤：剔除所有plant_type为'tree'的植物 ---
initial_rows = len(df)
df_filtered = df[df['plant_type'] != 'tree'].copy() # 使用.copy()避免SettingWithCopyWarning
rows_after_filter = len(df_filtered)
print(f"筛选前共有 {initial_rows} 行数据。")
print(f"筛选掉 {initial_rows - rows_after_filter} 种树木后，剩余 {rows_after_filter} 行数据。")


# --- 终处理与保存 ---
# 定义最终输出到CSV文件中的列，注意'plant_type'列在筛选后就不再需要了
final_columns = [
    "general_plant_id",
    "sunlight",
    "watering",
    "drought_tolerant",
    "absolute_min_temp_c"
]
df_final = df_filtered[final_columns]

# 为了保持数据一致性，按ID排序
df_final = df_final.sort_values(by="general_plant_id").reset_index(drop=True)

# 【修改之处】移除了之前对sunlight列的转换，因为这个操作已经提前到process_plant_details函数中
# df_final['sunlight'] = df_final['sunlight'].apply(lambda x: json.dumps(x, ensure_ascii=False))

# 确保数据类型正确
df_final["general_plant_id"] = pd.to_numeric(df_final["general_plant_id"]).astype("Int64")
df_final["drought_tolerant"] = df_final["drought_tolerant"].astype(bool)


# 创建输出目录（如果不存在）
os.makedirs(os.path.dirname(OUTPUT_PATH), exist_ok=True)

# 将最终的DataFrame保存为CSV文件，不包含索引列
df_final.to_csv(OUTPUT_PATH, index=False)

print("\n处理完成！")
print(f"Table13已成功保存至: \n{OUTPUT_PATH}")

# 显示最终表格的前几行
df_final.head()

筛选前共有 1484 行数据。
筛选掉 476 种树木后，剩余 1008 行数据。

处理完成！
Table13已成功保存至: 
02_wrangled_data/Table13_GeneralPlantListforRecommendation.csv


,general_plant_id,sunlight,watering,drought_tolerant,absolute_min_temp_c
0,398,"[""full sun"", ""part shade""]",Average,True,-17.8
1,399,"[""full sun"", ""part shade""]",Average,False,-23.3
2,400,"[""full sun"", ""part shade""]",Average,True,-28.9
3,401,"[""Full sun"", ""part shade""]",Average,False,-23.3
4,402,"[""full sun"", ""part shade""]",Average,True,-28.9


## Step 6 - Import Plant Disease Link Table (Table13) into MySQL

In [19]:
import mysql.connector
from mysql.connector import Error

# 数据库连接配置 (与您提供的一致)
# 注意：在生产环境中，请勿将密码硬编码在代码中。
db_config = {
    'host': 'database-plantx.cqz06uycysiz.us-east-1.rds.amazonaws.com',
    'user': 'zihan',
    'password': '2002317Yzh12138.',
    'database': 'FIT5120_PlantX_Database',
    'allow_local_infile': True,
    'use_pure': True,
    'charset': 'utf8mb4'
}

# 尝试连接数据库并创建Table13
try:
    connection = mysql.connector.connect(**db_config)
    if connection.is_connected():
        print("成功连接到MySQL服务器。")
        cursor = connection.cursor()

        # SQL语句：创建 Table13_GeneralPlantListforRecommendation
        # 注意我们最终的CSV中不包含 plant_type 列，所以建表语句也与之对应
        create_table_13 = """
        CREATE TABLE IF NOT EXISTS Table13_GeneralPlantListforRecommendation (
            general_plant_id INT PRIMARY KEY,
            sunlight JSON,
            watering TEXT,
            drought_tolerant BOOLEAN,
            absolute_min_temp_c DOUBLE
        ) CHARACTER SET utf8mb4 COLLATE utf8mb4_unicode_ci;
        """

        cursor.execute(create_table_13)
        connection.commit()
        print("Table13 的表结构创建成功。")

except Error as e:
    print(f"创建Table13表结构时发生错误: {e}")

finally:
    if 'connection' in locals() and connection.is_connected():
        cursor.close()
        connection.close()
        print("用于创建Table13表结构的MySQL连接已关闭。")

成功连接到MySQL服务器。
Table13 的表结构创建成功。
用于创建Table13表结构的MySQL连接已关闭。


## Step 7 - Import Data into Table13

In [20]:
try:
    # 重新建立连接以进行数据导入
    connection = mysql.connector.connect(**db_config)
    if connection.is_connected():
        print("为导入Table13数据，已成功连接到MySQL服务器。")
        cursor = connection.cursor()

        # 定义从CSV文件加载数据的查询语句
        # 注意：Pandas默认使用 '\n' 作为换行符，比'\r\n'更通用
        load_data_query_13 = f"""
        LOAD DATA LOCAL INFILE '02_wrangled_data/Table13_GeneralPlantListforRecommendation.csv'
        INTO TABLE Table13_GeneralPlantListforRecommendation
        CHARACTER SET utf8mb4
        FIELDS TERMINATED BY ','
        OPTIONALLY ENCLOSED BY '"'
        LINES TERMINATED BY '\\n'
        IGNORE 1 LINES
        (
            general_plant_id,
            sunlight,
            watering,
            @drought_tolerant_var,
            absolute_min_temp_c
        )
        SET drought_tolerant = IF(@drought_tolerant_var = 'True', 1, 0);
        """

        cursor.execute(load_data_query_13)
        connection.commit()
        print(f"Table13 数据导入成功！影响行数: {cursor.rowcount}")

except Error as e:
    print(f"导入Table13数据时发生错误: {e}")

finally:
    if 'connection' in locals() and connection.is_connected():
        cursor.close()
        connection.close()
        print("用于导入Table13的MySQL连接已关闭。")

为导入Table13数据，已成功连接到MySQL服务器。
Table13 数据导入成功！影响行数: 1008
用于导入Table13的MySQL连接已关闭。


## Step 8 - Verify Imported Rows and Preview (Table11)

In [21]:
try:
    connection = mysql.connector.connect(**db_config)
    if connection.is_connected():
        cursor = connection.cursor()

        # 获取表中的总行数
        cursor.execute("SELECT COUNT(*) FROM Table13_GeneralPlantListforRecommendation")
        row_count = cursor.fetchone()[0]
        print(f"Table13_GeneralPlantListforRecommendation 表中当前包含 {row_count} 行数据。")

        # 获取并打印前5行数据以供预览
        print("\n--- Table13 前5行数据预览 ---")
        cursor.execute("SELECT * FROM Table13_GeneralPlantListforRecommendation LIMIT 5")
        rows = cursor.fetchall()
        for row in rows:
            print(row)

except Error as e:
    print(f"验证Table13时发生错误: {e}")

finally:
    if 'connection' in locals() and connection.is_connected():
        cursor.close()
        connection.close()
        print("用于验证Table13的MySQL连接已关闭。")

Table13_GeneralPlantListforRecommendation 表中当前包含 1008 行数据。

--- Table13 前5行数据预览 ---
(398, '["full sun", "part shade"]', 'Average', 1, -17.8)
(399, '["full sun", "part shade"]', 'Average', 0, -23.3)
(400, '["full sun", "part shade"]', 'Average', 1, -28.9)
(401, '["Full sun", "part shade"]', 'Average', 0, -23.3)
(402, '["full sun", "part shade"]', 'Average', 1, -28.9)
用于验证Table13的MySQL连接已关闭。
